In [1]:
import numpy as np
from RBM import RBM
import torchvision.datasets as datasets
import torch
from data_cleaning import *

In [2]:
mnist_trainset = datasets.MNIST(root='./data', train=True, download=True, transform=None)
mnist_testset = datasets.MNIST(root='./data', train=False, download=True, transform=None)

In [3]:
size_dataset = mnist_trainset.test_data.shape[0]
v_data = mnist_trainset.train_data.reshape(size_dataset, -1).float()
v_data.size()

/home/karelgeraedts/Documents/uni/Master_Theses/code/Renormalization-Group-Bialek/.venv/lib/python3.10/site-packages/torchvision/datasets/mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
/home/karelgeraedts/Documents/uni/Master_Theses/code/Renormalization-Group-Bialek/.venv/lib/python3.10/site-packages/torchvision/datasets/mnist.py:75: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


torch.Size([60000, 784])

In [4]:
def normalize_data(v_data):
    stds = torch.std(v_data, dim=0)
    print(torch.argwhere(stds == 0).numpy())
    return (v_data - torch.mean(v_data, dim=0)) / torch.std(v_data, dim=0)

In [18]:
# Load Ebo's dataset
f = "data/HG19_k6_datasets/HG19_k6_mu0.00_(20,4,5)_B0.55_N1000_00.dat"
v_data = read_input(f)
v_data = torch.tensor(v_data).float()
v_data.size(), v_data.type()

(torch.Size([1000, 20]), 'torch.FloatTensor')

In [19]:
# Initialize
epochs = 100
n_visible = v_data.shape[1]
n_hidden = 5
lr = 0.01
CD_depth = 5
batch_size = 50

# Normalize data s.t. N(0, 1)
#v_data = normalize_data(v_data)

# Create RBM layer
rbm = RBM(n_visible, n_hidden, CD_depth)

# Optimizer
optimizer = torch.optim.SGD(rbm.parameters(), lr=lr)

# Things to keep track of
loss_ = []

for epoch in range(epochs):
    # Randomly permute the dataset
    v_data = v_data[torch.randperm(v_data.size()[0])]

    for i in range(0, len(v_data), batch_size):
        # Get batch
        v_batch = v_data[i:i+batch_size]

        # Forward
        h_batch, _ = rbm.v_to_h(v_batch)
        v_forward = rbm(v_batch)
        h_forward, _ = rbm.v_to_h(v_forward)

        # Zero gradients
        optimizer.zero_grad()

        # Compute loss
        loss = -1 * ((v_batch.T @ h_batch).mean() - (v_forward.T @ h_forward).mean())
        loss_.append(loss)
        
        # # Backward
        # loss.backward(retain_graph=True)

        # Update weights
        optimizer.step()

    # Print loss
    if (epoch+1) % 10 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 100, loss.item()))


Epoch [10/100], Loss: 10.2100
Epoch [20/100], Loss: 7.7600
Epoch [30/100], Loss: 9.5200
Epoch [40/100], Loss: 9.3700
Epoch [50/100], Loss: 9.6300
Epoch [60/100], Loss: 10.5400
Epoch [70/100], Loss: 10.5100
Epoch [80/100], Loss: 11.7200
Epoch [90/100], Loss: 8.8100
Epoch [100/100], Loss: 10.9600


In [16]:
for param in rbm.parameters():
    print(param)

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       requires_grad=True)
Parameter containing:
tensor([0., 0., 0., 0., 0.], requires_grad=True)
